In [ ]:
from __future__ import print_function, unicode_literals
import contextlib
import datetime
import io
import re
import sys

from fabric import api, state, network

In [ ]:
class WriteOnlyBuffer(object):
    def write(self, to_discard):
        pass

@contextlib.contextmanager
def nostdout():
    "plz shut up fabric"
    save_stdout = sys.stdout
    sys.stdout = WriteOnlyBuffer()
    yield
    sys.stdout = save_stdout

def scp_get(host, filename):
    buf = io.BytesIO()

    api.env.use_ssh_config = True
    api.env.host_string = host

    with nostdout(), api.hide('output', 'running', 'warnings'), api.settings(warn_only=True):
        api.get(filename, local_path=buf)
        network.disconnect_all()

    return buf.getvalue().decode('ascii').splitlines()

In [ ]:
NOVA_QUOTA_PARTS = {'instances', 'cores', 'ram', 'floating-ips', 'security-groups'}

def parse_nova_quotas(args):
    args = args.split()
    project = args.pop()
    assert len(args) % 2 == 0 # even
    
    iargs = iter(args)
    quota_update = {}
    for part in iargs:
        part = part.lstrip('--')
        assert part in NOVA_QUOTA_PARTS
        value = int(next(iargs))
        quota_update['nova:' + part] = value
    
    return project, quota_update

In [ ]:
NEUTRON_QUOTA_PARTS = {'port', 'floatingip', 'tenant_id'}

def parse_neutron_quotas(args):
    args = args.split()
    assert len(args) % 2 == 0 # even
    
    iargs = iter(args)
    quota_update = {}
    project = None
    for part in iargs:
        part = part.lstrip('--')
        assert part in NEUTRON_QUOTA_PARTS
        value = next(iargs)
        if part == 'tenant_id':
            project = value
        else:
            quota_update['neutron:' + part] = int(value)
    
    assert project is not None
    return project, quota_update

In [ ]:
def parse_quotas(lines):
    quotas = {}
    ticket_context = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('#'):
            result = re.search(r'ticket\s*(?P<ticket_num>[0-9]+)', line, flags=re.IGNORECASE)
            if result:
                ticket_context = result.groupdict()['ticket_num']
            continue

        service, command, args = line.split(' ', 2) # maxsplit=2

        # we only know how to handle...
        assert service in ['neutron', 'nova']
        assert command == 'quota-update'

        project, quota_update = {'neutron': parse_neutron_quotas, 'nova': parse_nova_quotas}[service](args)
        if project in quotas:
            quotas[project].update(quota_update)
        else:
            quotas[project] = quota_update

    return quotas

In [ ]:
host = 'm01-03'
filename = '/root/quota-update.sh'

quota_lines = scp_get(host, filename)
print('Report generated at {} from {} on {}'.format(datetime.datetime.now().isoformat(), filename, host))

quotas = parse_quotas(quota_lines)
columns = (
    ['nova:' + p for p in NOVA_QUOTA_PARTS]
    + ['neutron:' + p for p in NEUTRON_QUOTA_PARTS]
)
print('{},{}'.format('project', ','.join(columns)))
for project, quota in quotas.items():
    print('{},{}'.format(project, ','.join(str(quota.get(key, '')) for key in sorted(columns))))
print()